# Arc LLM: Text to 3D Visualization (Google Colab Runner)

This notebook allows you to run Arc LLM directly in Google Colab. It will clone the repository from GitHub and set up everything you need to convert text descriptions into 3D visualizations.

## 1. Clone the Repository and Install Dependencies

First, let's clone the Arc LLM repository and install the required dependencies.

In [ ]:
# Clone the Arc LLM repository
!git clone https://github.com/lillylight/arc-llm.git
%cd arc-llm

In [ ]:
# Install dependencies
!pip install -r requirements.txt

In [ ]:
# Install required dependencies
!pip install torch numpy scipy einops rerun-sdk

## 2. Set Claude API Key

To use the Claude API, you need to provide an API key. You can get an API key from [Anthropic](https://www.anthropic.com/).

In [ ]:
import os
from google.colab import userdata

# Try to get the API key from Colab secrets
try:
    CLAUDE_API_KEY = userdata.get('CLAUDE_API_KEY')
    print("Claude API key loaded from Colab secrets.")
except Exception as e:
    CLAUDE_API_KEY = None
    print("Claude API key not found in Colab secrets.")

# If not found in secrets, prompt the user to enter it
if not CLAUDE_API_KEY:
    CLAUDE_API_KEY = input("Enter your Claude API key: ")

# Set the API key as an environment variable
os.environ["CLAUDE_API_KEY"] = CLAUDE_API_KEY

## 3. Import Arc LLM

Now let's import the Arc LLM module and create an instance of the ArcLLM class.

In [ ]:
import sys
import os

# Add the current directory to the Python path
sys.path.append(os.path.abspath("."))

# Create a symbolic link for the spatiallm module
!mkdir -p spatiallm
!touch spatiallm/__init__.py
!echo "from simplified_spatiallm import Layout, Wall, Door, Window, Bbox, visualize_layout" > spatiallm/__init__.py

# Import Arc LLM
from arc_llm import ArcLLM

# Create an instance of ArcLLM
arc_llm = ArcLLM(claude_api_key=CLAUDE_API_KEY)

## 4. Generate Layout from Text

Now let's generate a layout from a text description. You can choose between a simple room example or a complex house example.

In [ ]:
# Example selection
from ipywidgets import widgets
from IPython.display import display

example_type = widgets.Dropdown(
    options=['Simple Room', 'Complex House'],
    value='Simple Room',
    description='Example:',
    disabled=False,
)

display(example_type)

# Load example files
with open("examples/living_room.txt", "r") as f:
    simple_room_text = f.read()
    
with open("examples/complex_house.txt", "r") as f:
    complex_house_text = f.read()

# Function to get selected example text
def get_example_text():
    if example_type.value == 'Simple Room':
        return simple_room_text
    else:
        return complex_house_text

# Display the selected example
text = get_example_text()
print(f"Text description:\n{text}")

In [ ]:
# Generate layout from text
layout_str = arc_llm.convert_text_to_layout(text)
print(f"Generated layout:\n{layout_str}")

## 5. Visualize Layout

Now let's visualize the layout using rerun. We'll save the visualization to a file and then display it in the notebook.

In [ ]:
# Save the visualization to a file
output_file = "visualization.rrd"
arc_llm.visualize(layout_str, output_file=output_file)

In [ ]:
# Install rerun viewer if not already installed
!pip install rerun-viewer

In [ ]:
# View the visualization
!rerun visualization.rrd

## 6. Alternative: Use rerun-sdk to display the visualization in the notebook

If the above method doesn't work, we can use rerun-sdk to display the visualization directly in the notebook.

In [ ]:
import rerun as rr
import rerun.blueprint as rrb
from IPython.display import IFrame

# Start a rerun server
rr.init("rerun_arcllm")
rr.serve()

In [ ]:
# Display the visualization in an iframe
IFrame(src="http://localhost:9876", width=800, height=600)

## 7. Web Interface (Optional)

If you want to use the web interface, you can run the following cells to start it.

In [ ]:
# Install additional dependencies for the web interface
!pip install flask pyngrok

In [ ]:
# Import the web interface
from web_interface import app
from pyngrok import ngrok

# Start ngrok
public_url = ngrok.connect(5000)
print(f"Web interface available at: {public_url}")

In [ ]:
# Run the Flask app
app.run()